### Getting Started Cohere

In [ ]:
import cohere
api_key = '<your-api-key>'

co = cohere.Client(api_key)
co.check_api_key()

#### Co.Generate

In [ ]:
res_generate = co.generate(
    prompt='Hello, how'
)

print(res_generate)

Nightly 版を使用し、最大 50 トークン含むテキストを 3 つ生成する

In [ ]:
res_generate = co.generate(
    prompt='Hello, how',
    model='command-nightly',
    num_generations=3,
    max_tokens=50
)

print(res_generate)

最も可能性の高い上位 5 個(k=5)のトークンのみを生成対象として考慮し、生成におけるランダム性を最大(temperature=5.0)にする

In [ ]:
res_generate = co.generate(
    prompt='Hello, how',
    model='command-nightly',
    num_generations=3,
    max_tokens=50,
    k=5,
    temperature=5.0
)

print(res_generate)

生成結果を JSON stream で受け取る（レスポンスの内容を少しずつレンダリングする UI にとって有益らしい）

In [ ]:
res_generate_streaming = co.generate(
    prompt='Hello, how',
    stream=True
)

for index, token in enumerate(res_generate_streaming):
    print(f"{index}: {token}")

res_generate_streaming.texts

#### Co.Embed

In [ ]:
res_embed = co.embed(
    texts=['hello', 'world']
)

print(res_embed)

軽量版モデル（model=embed-english-light-v2.0）を使ってみる

In [ ]:
res_embed = co.embed(
    texts=['hello', 'world'],
    model='embed-english-light-v2.0'
)

print(res_embed)

multilingual 対応のモデル（model=embed-multilingual-v2.0）を使ってみる

In [ ]:
res_embed = co.embed(
    texts=['こんにちは', '世界'],
    model='embed-multilingual-v2.0'
)

print(res_embed)

#### Co.Classify

In [ ]:
from cohere.responses.classify import Example

examples=[
  Example("Dermatologists don't like her!", "Spam"),
  Example("'Hello, open to this?'", "Spam"),
  Example("I need help please wire me $1000 right now", "Spam"),
  Example("Nice to know you ;)", "Spam"),
  Example("Please help me?", "Spam"),
  Example("Your parcel will be delivered today", "Not spam"),
  Example("Review changes to our Terms and Conditions", "Not spam"),
  Example("Weekly sync notes", "Not spam"),
  Example("'Re: Follow up from today's meeting'", "Not spam"),
  Example("Pre-read for tomorrow", "Not spam"),
]

inputs=[
  "Confirm your email address",
  "hey i need u to send some $",
]

res_classify = co.classify(
    inputs=inputs,
    examples=examples,
)

print(res_classify)

軽量版モデル（model=embed-english-light-v2.0）で最大トークン長より長い入力を渡した際は、エラーが返却される（truncate=NONE）ようにする

In [ ]:
inputs_max_exceeded=[
  "Confirm your email address",
  "hey i need u to send some $",
  "Please sign the receipt attached for the arrival of new office facilities.",
  "Attached is the paper concerning with the cancellation of your current credit card.",
  "The monthly financial statement is attached within the email.",
  "The audit report you inquired is attached in the mail. Please review and transfer it to the related department.",
  "There is a message for you from 01478719696, on 2016/08/23 18:26:30. You might want to check it when you get a chance.Thanks!",
  "Dear John, I am attaching the mortgage documents relating to your department. They need to be signed in urgent manner.",
  "This email/fax transmission is confidential and intended solely for the person or organisation to whom it is addressed. If you are not the intended recipient, you must not copy, distribute or disseminate the information, or take any action in reliance of it. Any views expressed in this message are those of the individual sender, except where the sender specifically states them to be the views of any organisation or employer. If you have received this message in error, do not open any attachment but please notify the sender (above) deleting this message from your system. For email transmissions please rely on your own virus check no responsibility is taken by the sender for any damage rising out of any bug or virus infection.",
  "Sent with Genius Scan for iOS. http://bit.ly/download-genius-scan",
  "Hey John, as you requested, attached is the paycheck for your next month?s salary in advance.",
  "I am sending you the flight tickets for your business conference abroad next month. Please see the attached and note the date and time.",
  "Attached is the bank transactions made from the company during last month. Please file these transactions into financial record.",
  "example.com SV9100 InMail",
  "Attached file is scanned image in PDF format. Use Acrobat(R)Reader(R) or Adobe(R)Reader(R) of Adobe Systems Incorporated to view the document. Adobe(R)Reader(R) can be downloaded from the following URL: Adobe, the Adobe logo, Acrobat, the Adobe PDF logo, and Reader are registered trademarks or trademarks of Adobe Systems Incorporated in the United States and other countries.",
  "Here is the travel expense sheet for your upcoming company field trip. Please write down the approximate costs in the attachment.",
  "Attached is the list of old office facilities that need to be replaced. Please copy the list into the purchase order form.",
  "We are sending you the credit card receipt from yesterday. Please match the card number and amount.",
  "Queries to: <scanner@example.com",
  "Please find our invoice attached.",
  "Hello! We are looking for employees working remotely. My name is Margaret, I am the personnel manager of a large International company. Most of the work you can do from home, that is, at a distance. Salary is $3000-$6000. If you are interested in this offer, please visit",
  "Please see the attached copy of the report.",
  "Attached is a pdf file containing items that have shipped. Please contact us if there are any questions or further assistance we can provide",
  "Please see the attached license and send it to the head office.",
  "Please find attached documents as requested.",
  "We are looking for employees working remotely. My name is Enid, I am the personnel manager of a large International company. Most of the work you can do from home, that is, at a distance. Salary is $2000-$5400. If you are interested in this offer, please visit",
  "Your item #123456789 has been sent to you by carrier. He will arrive to youon 23th of September, 2016 at noon.",
  "Dear John, we are very sorry to inform you that the item you requested is out of stock. Here is the list of items similar to the ones you requested. Please take a look and let us know if you would like to substitute with any of them.",
  "Thank you for your order! Please find your final sales receipt attached to this email. Your USPS Tracking Number is: 123456789. This order will ship tomorrow and you should be able to begin tracking tomorrow evening after it is picked up. If you have any questions or experience any problems, please let us know so we can assist you. Thanks again and enjoy! Thanks,",
  "Thanks for using electronic billing. Please find your document attached",
  "Dear John, I attached the clients’ accounts for your next operation. Please look through them and collect their data. I expect to hear from you soon.",
  "You are receiving this email because the company has assigned you as part of the approval team. Please review the attached proposal form and make your approval decision. If you have any problem regarding the submission, please contact Gonzalo.",
  "this is to inform you that your Debit Card is temporarily blocked as there were unknown transactions made today. We attached the scan of transactions. Please confirm whether you made these transactions.",
  "The letter is attached. Please review his concerns carefully and reply him as soon as possible. King regards,",
  "Confirm your email address",
  "hey i need u to send some $",
  "Please sign the receipt attached for the arrival of new office facilities.",
  "Attached is the paper concerning with the cancellation of your current credit card.",
  "The monthly financial statement is attached within the email.",
  "The audit report you inquired is attached in the mail. Please review and transfer it to the related department.",
  "There is a message for you from 01478719696, on 2016/08/23 18:26:30. You might want to check it when you get a chance.Thanks!",
  "Dear John, I am attaching the mortgage documents relating to your department. They need to be signed in urgent manner.",
  "This email/fax transmission is confidential and intended solely for the person or organisation to whom it is addressed. If you are not the intended recipient, you must not copy, distribute or disseminate the information, or take any action in reliance of it. Any views expressed in this message are those of the individual sender, except where the sender specifically states them to be the views of any organisation or employer. If you have received this message in error, do not open any attachment but please notify the sender (above) deleting this message from your system. For email transmissions please rely on your own virus check no responsibility is taken by the sender for any damage rising out of any bug or virus infection.",
  "Sent with Genius Scan for iOS. http://bit.ly/download-genius-scan",
  "Hey John, as you requested, attached is the paycheck for your next month?s salary in advance.",
  "I am sending you the flight tickets for your business conference abroad next month. Please see the attached and note the date and time.",
  "Attached is the bank transactions made from the company during last month. Please file these transactions into financial record.",
  "example.com SV9100 InMail",
  "Attached file is scanned image in PDF format. Use Acrobat(R)Reader(R) or Adobe(R)Reader(R) of Adobe Systems Incorporated to view the document. Adobe(R)Reader(R) can be downloaded from the following URL: Adobe, the Adobe logo, Acrobat, the Adobe PDF logo, and Reader are registered trademarks or trademarks of Adobe Systems Incorporated in the United States and other countries.",
  "Here is the travel expense sheet for your upcoming company field trip. Please write down the approximate costs in the attachment.",
  "Attached is the list of old office facilities that need to be replaced. Please copy the list into the purchase order form.",
  "We are sending you the credit card receipt from yesterday. Please match the card number and amount.",
  "Queries to: <scanner@example.com",
  "Please find our invoice attached.",
  "Hello! We are looking for employees working remotely. My name is Margaret, I am the personnel manager of a large International company. Most of the work you can do from home, that is, at a distance. Salary is $3000-$6000. If you are interested in this offer, please visit",
  "Please see the attached copy of the report.",
  "Attached is a pdf file containing items that have shipped. Please contact us if there are any questions or further assistance we can provide",
  "Please see the attached license and send it to the head office.",
  "Please find attached documents as requested.",
  "We are looking for employees working remotely. My name is Enid, I am the personnel manager of a large International company. Most of the work you can do from home, that is, at a distance. Salary is $2000-$5400. If you are interested in this offer, please visit",
  "Your item #123456789 has been sent to you by carrier. He will arrive to youon 23th of September, 2016 at noon.",
  "Dear John, we are very sorry to inform you that the item you requested is out of stock. Here is the list of items similar to the ones you requested. Please take a look and let us know if you would like to substitute with any of them.",
  "Thank you for your order! Please find your final sales receipt attached to this email. Your USPS Tracking Number is: 123456789. This order will ship tomorrow and you should be able to begin tracking tomorrow evening after it is picked up. If you have any questions or experience any problems, please let us know so we can assist you. Thanks again and enjoy! Thanks,",
  "Thanks for using electronic billing. Please find your document attached",
  "Dear John, I attached the clients’ accounts for your next operation. Please look through them and collect their data. I expect to hear from you soon.",
  "You are receiving this email because the company has assigned you as part of the approval team. Please review the attached proposal form and make your approval decision. If you have any problem regarding the submission, please contact Gonzalo.",
  "this is to inform you that your Debit Card is temporarily blocked as there were unknown transactions made today. We attached the scan of transactions. Please confirm whether you made these transactions.",
  "The letter is attached. Please review his concerns carefully and reply him as soon as possible. King regards,",
  "Confirm your email address",
  "hey i need u to send some $",
  "Please sign the receipt attached for the arrival of new office facilities.",
  "Attached is the paper concerning with the cancellation of your current credit card.",
  "The monthly financial statement is attached within the email.",
  "The audit report you inquired is attached in the mail. Please review and transfer it to the related department.",
  "There is a message for you from 01478719696, on 2016/08/23 18:26:30. You might want to check it when you get a chance.Thanks!",
  "Dear John, I am attaching the mortgage documents relating to your department. They need to be signed in urgent manner.",
  "This email/fax transmission is confidential and intended solely for the person or organisation to whom it is addressed. If you are not the intended recipient, you must not copy, distribute or disseminate the information, or take any action in reliance of it. Any views expressed in this message are those of the individual sender, except where the sender specifically states them to be the views of any organisation or employer. If you have received this message in error, do not open any attachment but please notify the sender (above) deleting this message from your system. For email transmissions please rely on your own virus check no responsibility is taken by the sender for any damage rising out of any bug or virus infection.",
  "Sent with Genius Scan for iOS. http://bit.ly/download-genius-scan",
  "Hey John, as you requested, attached is the paycheck for your next month?s salary in advance.",
  "I am sending you the flight tickets for your business conference abroad next month. Please see the attached and note the date and time.",
  "Attached is the bank transactions made from the company during last month. Please file these transactions into financial record.",
  "example.com SV9100 InMail",
  "Attached file is scanned image in PDF format. Use Acrobat(R)Reader(R) or Adobe(R)Reader(R) of Adobe Systems Incorporated to view the document. Adobe(R)Reader(R) can be downloaded from the following URL: Adobe, the Adobe logo, Acrobat, the Adobe PDF logo, and Reader are registered trademarks or trademarks of Adobe Systems Incorporated in the United States and other countries.",
  "Here is the travel expense sheet for your upcoming company field trip. Please write down the approximate costs in the attachment.",
  "Attached is the list of old office facilities that need to be replaced. Please copy the list into the purchase order form.",
  "We are sending you the credit card receipt from yesterday. Please match the card number and amount.",
  "Queries to: <scanner@example.com",
  "Please find our invoice attached.",
  "Hello! We are looking for employees working remotely. My name is Margaret, I am the personnel manager of a large International company. Most of the work you can do from home, that is, at a distance. Salary is $3000-$6000. If you are interested in this offer, please visit",
  "Please see the attached copy of the report.",
  "Attached is a pdf file containing items that have shipped. Please contact us if there are any questions or further assistance we can provide",
  "Please see the attached license and send it to the head office.",
  "Please find attached documents as requested.",
  "We are looking for employees working remotely. My name is Enid, I am the personnel manager of a large International company. Most of the work you can do from home, that is, at a distance. Salary is $2000-$5400. If you are interested in this offer, please visit",
  "Your item #123456789 has been sent to you by carrier. He will arrive to youon 23th of September, 2016 at noon.",
  "Dear John, we are very sorry to inform you that the item you requested is out of stock. Here is the list of items similar to the ones you requested. Please take a look and let us know if you would like to substitute with any of them.",
  "Thank you for your order! Please find your final sales receipt attached to this email. Your USPS Tracking Number is: 123456789. This order will ship tomorrow and you should be able to begin tracking tomorrow evening after it is picked up. If you have any questions or experience any problems, please let us know so we can assist you. Thanks again and enjoy! Thanks,",
  "Thanks for using electronic billing. Please find your document attached",
  "Dear John, I attached the clients’ accounts for your next operation. Please look through them and collect their data. I expect to hear from you soon.",
  "You are receiving this email because the company has assigned you as part of the approval team. Please review the attached proposal form and make your approval decision. If you have any problem regarding the submission, please contact Gonzalo.",
  "this is to inform you that your Debit Card is temporarily blocked as there were unknown transactions made today. We attached the scan of transactions. Please confirm whether you made these transactions.",
  "The letter is attached. Please review his concerns carefully and reply him as soon as possible. King regards,",
  "Confirm your email address",
  "hey i need u to send some $",
  "Please sign the receipt attached for the arrival of new office facilities.",
  "Attached is the paper concerning with the cancellation of your current credit card.",
  "The monthly financial statement is attached within the email.",
  "The audit report you inquired is attached in the mail. Please review and transfer it to the related department.",
  "There is a message for you from 01478719696, on 2016/08/23 18:26:30. You might want to check it when you get a chance.Thanks!",
  "Dear John, I am attaching the mortgage documents relating to your department. They need to be signed in urgent manner.",
  "This email/fax transmission is confidential and intended solely for the person or organisation to whom it is addressed. If you are not the intended recipient, you must not copy, distribute or disseminate the information, or take any action in reliance of it. Any views expressed in this message are those of the individual sender, except where the sender specifically states them to be the views of any organisation or employer. If you have received this message in error, do not open any attachment but please notify the sender (above) deleting this message from your system. For email transmissions please rely on your own virus check no responsibility is taken by the sender for any damage rising out of any bug or virus infection.",
  "Sent with Genius Scan for iOS. http://bit.ly/download-genius-scan",
  "Hey John, as you requested, attached is the paycheck for your next month?s salary in advance.",
  "I am sending you the flight tickets for your business conference abroad next month. Please see the attached and note the date and time.",
  "Attached is the bank transactions made from the company during last month. Please file these transactions into financial record.",
  "example.com SV9100 InMail",
  "Attached file is scanned image in PDF format. Use Acrobat(R)Reader(R) or Adobe(R)Reader(R) of Adobe Systems Incorporated to view the document. Adobe(R)Reader(R) can be downloaded from the following URL: Adobe, the Adobe logo, Acrobat, the Adobe PDF logo, and Reader are registered trademarks or trademarks of Adobe Systems Incorporated in the United States and other countries.",
  "Here is the travel expense sheet for your upcoming company field trip. Please write down the approximate costs in the attachment.",
  "Attached is the list of old office facilities that need to be replaced. Please copy the list into the purchase order form.",
  "We are sending you the credit card receipt from yesterday. Please match the card number and amount.",
  "Queries to: <scanner@example.com",
  "Please find our invoice attached.",
  "Hello! We are looking for employees working remotely. My name is Margaret, I am the personnel manager of a large International company. Most of the work you can do from home, that is, at a distance. Salary is $3000-$6000. If you are interested in this offer, please visit",
  "Please see the attached copy of the report.",
  "Attached is a pdf file containing items that have shipped. Please contact us if there are any questions or further assistance we can provide",
  "Please see the attached license and send it to the head office.",
  "Please find attached documents as requested.",
  "We are looking for employees working remotely. My name is Enid, I am the personnel manager of a large International company. Most of the work you can do from home, that is, at a distance. Salary is $2000-$5400. If you are interested in this offer, please visit",
  "Your item #123456789 has been sent to you by carrier. He will arrive to youon 23th of September, 2016 at noon.",
  "Dear John, we are very sorry to inform you that the item you requested is out of stock. Here is the list of items similar to the ones you requested. Please take a look and let us know if you would like to substitute with any of them.",
  "Thank you for your order! Please find your final sales receipt attached to this email. Your USPS Tracking Number is: 123456789. This order will ship tomorrow and you should be able to begin tracking tomorrow evening after it is picked up. If you have any questions or experience any problems, please let us know so we can assist you. Thanks again and enjoy! Thanks,",
  "Thanks for using electronic billing. Please find your document attached",
  "Dear John, I attached the clients’ accounts for your next operation. Please look through them and collect their data. I expect to hear from you soon.",
  "You are receiving this email because the company has assigned you as part of the approval team. Please review the attached proposal form and make your approval decision. If you have any problem regarding the submission, please contact Gonzalo.",
  "this is to inform you that your Debit Card is temporarily blocked as there were unknown transactions made today. We attached the scan of transactions. Please confirm whether you made these transactions.",
  "The letter is attached. Please review his concerns carefully and reply him as soon as possible. King regards,",
]

res_classify = co.classify(
    inputs=inputs_max_exceeded,
    examples=examples,
    model='embed-english-light-v2.0',
    truncate='END'
)

res_classify.classifications

#### Co.Tokenize

In [ ]:
res_tokenize = co.tokenize(
    text='tokenize me !D',
)

print(res_tokenize)

Co.Detokenize

In [ ]:
tokens = [10002, 2261, 2012, 3362, 43] # tokenize me !D

res_detokenize = co.detokenize(
    tokens=tokens
)

print(res_detokenize)

#### Co.Detect_language

In [ ]:
res_detect_language = co.detect_language(
    texts=['Hello world', "'Здравствуй, Мир'", 'こんにちは世界', '世界你好', '안녕하세요']
)

res_detect_language

#### Co.summarize

In [ ]:
text = (
  "Ice cream is a sweetened frozen food typically eaten as a snack or dessert. "
  "It may be made from milk or cream and is flavoured with a sweetener, "
  "either sugar or an alternative, and a spice, such as cocoa or vanilla, "
  "or with fruit such as strawberries or peaches. "
  "It can also be made by whisking a flavored cream base and liquid nitrogen together. "
  "Food coloring is sometimes added, in addition to stabilizers. "
  "The mixture is cooled below the freezing point of water and stirred to incorporate air spaces "
  "and to prevent detectable ice crystals from forming. The result is a smooth, "
  "semi-solid foam that is solid at very low temperatures (below 2 °C or 35 °F). "
  "It becomes more malleable as its temperature increases.\n\n"
  "The meaning of the name \"ice cream\" varies from one country to another. "
  "In some countries, such as the United States, \"ice cream\" applies only to a specific variety, "
  "and most governments regulate the commercial use of the various terms according to the "
  "relative quantities of the main ingredients, notably the amount of cream. "
  "Products that do not meet the criteria to be called ice cream are sometimes labelled "
  "\"frozen dairy dessert\" instead. In other countries, such as Italy and Argentina, "
  "one word is used fo\r all variants. Analogues made from dairy alternatives, "
  "such as goat's or sheep's milk, or milk substitutes "
  "(e.g., soy, cashew, coconut, almond milk or tofu), are available for those who are "
  "lactose intolerant, allergic to dairy protein or vegan."
)

co_summarize = co.summarize(
    text=text
)

print(co_summarize)

長さを短く（length=short）し、箇条書き形式（format=bullets）で出力する

In [ ]:
co_summarize = co.summarize(
    text=text,
    length='short',
    format='bullets'
)

print(co_summarize)

軽量版のモデルを使い（model=command-light）、元の文章からの抽出度を高くし（extractiveness=high）、箇条書き形式で出力する

In [ ]:
co_summarize = co.summarize(
    text=text,
    model='command-light',
    length='short',
    format='bullets',
    extractiveness='high'
)

print(co_summarize)

#### Co.Rerank

In [ ]:
docs = [
    'Carson City is the capital city of the American state of Nevada.',
    'The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean. Its capital is Saipan.',
    'Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district.',
    'Capital punishment (the death penalty) has existed in the United States since beforethe United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states.'
]

co_rerank = co.rerank(
    model='rerank-english-v2.0',
    query='What is the capital of the United States?',
    documents=docs
)

print(co_rerank)

日本語で試してみる（model=rerank-multilingual-v2.0）

In [ ]:
docs_ja = [
    'カーソンシティはアメリカ・ネバダ州の州都である',
    '北マリアナ諸島連邦は太平洋に浮かぶ島々である。首都はサイパンである。',
    ' ワシントンD.C.（Washington, D.C.）は、アメリカ合衆国の首都である。連邦区である',
    '死刑制度はアメリカ合衆国が国である以前から存在する。2017年現在、死刑は50州のうち30州で合法である。'
]

co_rerank_ja = co.rerank(
    model='rerank-multilingual-v2.0',
    query='アメリカの首都は何ですか？',
    documents=docs_ja
)

print(co_rerank_ja)

日本語（model=rerank-multilingual-v2.0）で最も相関の高いもののみ（top_n=1）を返却する。

In [ ]:
docs_ja = [
    'カーソンシティはアメリカ・ネバダ州の州都である',
    '北マリアナ諸島連邦は太平洋に浮かぶ島々である。首都はサイパンである。',
    ' ワシントンD.C.（Washington, D.C.）は、アメリカ合衆国の首都である。連邦区である',
    '死刑制度はアメリカ合衆国が国である以前から存在する。2017年現在、死刑は50州のうち30州で合法である。'
]

co_rerank_ja = co.rerank(
    model='rerank-multilingual-v2.0',
    query='アメリカの首都は何ですか？',
    documents=docs_ja,
    top_n=1,
)

print(co_rerank_ja)